In [23]:
import harmonica as hm
import xarray as xr
import rasterio
import rioxarray
from rasterio.transform import from_origin
import numpy as np
from affine import Affine
from pathlib import Path

In [21]:
def export_grd_to_netcdf(grd_path: str, output_path: str, variable_name: str) -> None:
    """
    Load an Oasis‑Montaj .grd, rename dims, attach LCC CRS, and export to NetCDF.

    Parameters
    ----------
    grd_path : str
        Full path to the input .grd file.
    output_path : str
        Full path (including .nc) for the output NetCDF.
    """
    # 1. Load raw grid
    da_raw = hm.load_oasis_montaj_grid(fname=grd_path)\
    
    # Print basic info
    print("Dimensions:", da_raw.dims)
    print("Shape:", da_raw.shape)
    print("Coordinates:", list(da_raw.coords))
    
    # Print all attributes
    print("\nAttributes:")
    for attr_name, attr_value in da_raw.attrs.items():
        print(f"  {attr_name}: {attr_value}")
    
    # Print data summary
    print("\nData Summary (ignoring NaNs):")
    print("  Min:", float(da_raw.min(skipna=True)))
    print("  Max:", float(da_raw.max(skipna=True)))
    print("  Mean:", float(da_raw.mean(skipna=True)))
    print("  Std Dev:", float(da_raw.std(skipna=True)))
    print("  % NaNs:", float(da_raw.isnull().sum() / da_raw.size * 100), "%")

    # 2. Rename dims & update coord metadata
    da = da_raw.rename({"easting": "x", "northing": "y"})
    da.coords["x"].attrs.update(
        {
            "standard_name": "projection_x_coordinate",
            "long_name": "Easting",
            "units": "m",
            "axis": "X",
        }
    )
    da.coords["y"].attrs.update(
        {
            "standard_name": "projection_y_coordinate",
            "long_name": "Northing",
            "units": "m",
            "axis": "Y",
        }
    )

    # 3. Define the CRS
    crs = xr.DataArray(
        0,
        attrs={
            "grid_mapping_name": "lambert_conformal_conic",
            "standard_parallel": [46.0, 60.0],
            "latitude_of_projection_origin": 44.0,
            "longitude_of_central_meridian": -68.5,
            "false_easting": 0.0,
            "false_northing": 0.0,
            "semi_major_axis": 6378137.0,
            "inverse_flattening": 298.257222101,
            "epsg_code": "EPSG:6622",
            "crs_wkt": "NAD83(CSRS) / Quebec Lambert",
            "units": "m",
        },
    )

    # 4. Wrap & link mapping
    ds = xr.Dataset({variable_name: da})
    ds["crs"] = crs
    ds[variable_name].attrs["grid_mapping"] = "crs"

    # 5. Export
    ds.to_netcdf(output_path)

    print(f"grd file successfully exported to {output_path}")

In [28]:
### Run function

input_grd = r"c:\Users\TyHow\MinersAI Dropbox\Science\Geo Data\Canada\Quebec\Geophysics\Spectrometry\Quebec_Thorium_75m_QL_F.grd"

p = Path(input_grd)
output_path = p.with_suffix(".nc")

variable_name = "thorium"

export_grd_to_netcdf(input_grd, output_path, variable_name)

Dimensions: ('northing', 'easting')
Shape: (24463, 21462)
Coordinates: ['easting', 'northing']

Attributes:
  n_bytes_per_element: 4
  sign_flag: 2
  shape_e: 21462
  shape_v: 24463
  ordering: 1
  spacing_e: 75.0
  spacing_v: 75.0
  x_origin: -827925.0
  y_origin: 250950.0
  rotation: 0.0
  base_value: 0.0
  data_factor: 1.0
  map_projection: 0
  units_x: 0
  units_y: 0
  units_z: 0
  n_valid_points: 59501990
  grid_min: -7.9514875411987305
  grid_max: 73.64348602294922
  grid_median: 0.0
  grid_mean: 2.294743299484253
  grid_variance: 12.729840783305633
  process_flag: 0

Data Summary (ignoring NaNs):
  Min: -7.9514875411987305
  Max: 73.64348602294922
  Mean: 2.294743294487203
  Std Dev: 3.567890212626923
  % NaNs: 88.66682526485705 %
grd file successfully exported to c:\Users\TyHow\MinersAI Dropbox\Science\Geo Data\Canada\Quebec\Geophysics\Spectrometry\Quebec_Thorium_75m_QL_F.nc


In [ ]:
### Export to TIFF

# Path to your NetCDF and GeoTIFF output
netcdf_path = output_path
geotiff_output_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\QuebecProject\geo_data\Quebec_MAG_DV1_4326.tif"

# 1. Load the wrapped dataset and grab the layer
wrapped = xr.open_dataset(netcdf_path)
mag_da = wrapped[variable_name]]

# 2. Make sure x/y dims are set and original CRS is applied
mag_da = mag_da.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
mag_da = mag_da.rio.write_crs("EPSG:6622", inplace=False)

# 3. Reproject to WGS84 (EPSG:4326)
mag_da_4326 = mag_da.rio.reproject("EPSG:4326")

# 4. Export the reprojected layer as a GeoTIFF
mag_da_4326.rio.to_raster(geotiff_output_path)